In [2]:
import pandas as pd
import os
import json
from datetime import datetime
import numpy as np

In [3]:
!ls

Alito.csv                  README.md
Breyer.csv                 Roberts.csv
Data Creation.ipynb        Sotomayor.csv
Ginsburg.csv               Thomas.csv
Gorsuch.csv                decisions.csv
Kagan.csv                  transcripts_by_justice.csv
Kavanaugh.csv


In [35]:
files = ['Alito.csv', 'Breyer.csv', 'Ginsburg.csv', 'Gorsuch.csv', 
         'Kagan.csv', 'Kavanaugh.csv', 'Roberts.csv', 'Sotomayor.csv', 'Thomas.csv']

for file in files:
    df = pd.read_csv(file)
    df[['direction', 'majority']] = df[['direction', 'majority']].replace([1, 2], [0, 1])
    to_drop = df[(df['direction'].isnull())].index
    df = df.drop(labels=to_drop)
    df.to_csv(file, index=False)

In [63]:
for file in files:
    df = pd.read_csv(file)
    vote = []
    for i, row in df.iterrows():
        if row['partyWinning'] == 0.0 and row['majority'] == 0.0:
            vote.append(1)
        if row['partyWinning'] == 0.0 and row['majority'] == 1.0:
            vote.append(0)
        if row['partyWinning'] == 1.0 and row['majority'] == 0.0:
            vote.append(0)
        if row['partyWinning'] == 1.0 and row['majority'] == 1.0:
            vote.append(1)
    df['vote'] = vote
    df.to_csv(file, index=False)

In [64]:

df


,term,caseName,justiceId,speakerName,text,cleanText,duration,decidedBy,arguedOn,petitioner,respondent,docket,grantedOn,decidedOn,justice,partyWinning,direction,majority,vote
0,1991,United States v. Fordice,108,Clarence Thomas,"Mr. Goodman, is there any distinction between ...",mr goodman be there any distinction between a ...,52.23,Rehnquist Court,"Nov 13, 1991",United States,Fordice,90-1205,NaN,"Jun 26, 1992",108.0,1.0,0.0,0.0,0
1,1991,Evans v. United States,108,Clarence Thomas,"Why isn't it pertinent, Mr. Bryson? Our statut...",why be not it pertinent mr bryson our statute ...,37.48,Rehnquist Court,"Dec 9, 1991",Evans,United States,90-6105,NaN,"May 26, 1992",108.0,0.0,0.0,0.0,1
2,1991,Union Bank v. Wolas,108,Clarence Thomas,"Yes, Mr. Graham, if there... should we look at...",yes mr graham if there should we look at the f...,29.81,Rehnquist Court,"Nov 5, 1991",Union Bank,Wolas,90-1491,NaN,"Dec 11, 1991",108.0,1.0,0.0,0.0,0
3,1991,"Morales v. Trans World Airlines, Inc.",108,Clarence Thomas,"I'd like to ask you one question, counsel. You...",I would like to ask you one question counsel y...,49.42,Rehnquist Court,"Mar 3, 1992","Morales, Attorney General of Texas","Trans World Airlines, Inc.",90-1604,"Nov 27, 1991","Jun 1, 1992",108.0,0.0,0.0,0.0,1
4,1991,Hudson v. McMillian,108,Clarence Thomas,NaN,NaN,0.00,Rehnquist Court,"Nov 13, 1991",Hudson,McMillian,90-6531,NaN,"Feb 25, 1992",108.0,1.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1832,2018,Timbs v. Indiana,108,Clarence Thomas,NaN,NaN,0.00,Roberts Court,"Nov 28, 2018",Tyson Timbs,Indiana,17-1091,"Jun 18, 2018","Feb 20, 2019",108.0,1.0,0.0,0.0,0
1833,2018,"Lamps Plus, Inc. v. Varela",108,Clarence Thomas,NaN,NaN,0.00,Case pending,"Oct 29, 2018","Lamps Plus, Inc., et al.",Frank Varela,17-988,"Apr 30, 2018",NaN,108.0,1.0,0.0,0.0,0
1834,2018,Tennessee Wine and Spirits Retailers Associati...,108,Clarence Thomas,NaN,NaN,0.00,Case pending,"Jan 16, 2019",Tennessee Wine and Spirits Retailers Association,"Zackary W. Blair, Interim Director of the Tenn...",18-96,"Sep 27, 2018",NaN,108.0,0.0,0.0,0.0,1
1835,2018,Sturgeon v. Frost,108,Clarence Thomas,NaN,NaN,0.00,Case pending,"Nov 5, 2018",John Sturgeon,"Bert Frost, in His Official Capacity as Alaska...",17-949,"Jun 18, 2018",NaN,108.0,1.0,0.0,0.0,0


In [92]:
df = pd.DataFrame(columns=['term', 'caseName', 'justiceId', 'speakerName', 'text', 'cleanText',
                      'start', 'stop', 'duration', 'decidedBy', 'arguedOn', 
                      'petitioner', 'respondent', 'docket', 'grantedOn', 
                      'decidedOn'])

for i in range(2012, 2019):
    cases = os.listdir('cases/%s/' % str(i))
    print(i)
    for case in cases:
        with open('cases/%s/%s' % (str(i), case)) as f:
            file_contents = f.read()
            f.close()
        data = json.loads(file_contents)
        
        transcripts = data['caseTranscripts']
        if len(transcripts) == 0:
            continue
        
        term = data['term']
        caseName = data['caseName']
        decidedBy = data['decidedBy']
        arguedOn = data['arguedOn']
        petitioner = data['petitioner']
        respondent = data['respondent']
        docket = data['docket']
        grantedOn = data['grantedOn']
        decidedOn = data['decidedOn']

        
        for item in transcripts:
            transcript = item['transcript']
            for speaker in transcript:
                speakerName = speaker['speakerName']
                
                #only want transcript from justices
                if speakerName not in justices:
                    continue
                    
                justiceId = justice_ids[speakerName]
                
                for textObj in speaker['textObjs']:
                    text = textObj['text']
                    start = textObj['start']
                    stop = textObj['stop']
                    duration = textObj['duration']
                    cleanText = []
                    for sentence in textObj['cleanText']:
                        cleanText.extend(sentence)
                        
                    cleanText = ' '.join(cleanText)
                    
                    df = df.append({'term' : term, 
                                    'caseName' : caseName, 
                                    'justiceId' : justiceId, 
                                    'speakerName' : speakerName, 
                                    'text' : text, 
                                    'cleanText' : cleanText,
                                    'start' : start, 
                                    'stop' : stop,
                                    'duration' : duration,
                                    'decidedBy' : decidedBy,
                                    'arguedOn' : arguedOn, 
                                    'petitioner' : petitioner,
                                    'respondent' : respondent,
                                    'docket' : docket,
                                    'grantedOn' : grantedOn, 
                                    'decidedOn' : decidedOn},
                                    ignore_index=True)
                    
                    

2012
2013
2014
2015
2016
2017
2018


In [97]:
_91_11 = pd.read_csv('transcripts91_11.csv', index_col=0)

In [99]:
_12_18 = pd.read_csv('transcripts12_18.csv', index_col=0)

In [103]:
full_df = _91_11.append(_12_18)

In [104]:
full_df.to_csv('transcripts_by_justice.csv', index=False)

In [113]:
thomas_cases = full_df[full_df['justiceId'] == 108]['caseName'].unique()

In [120]:
case_text = ''

for item in full_df[(full_df['justiceId'] == 108) & (full_df['caseName'] == thomas_cases[0])]['text']:
    case_text += item + ' '

In [151]:
full_df.columns

Index(['term', 'caseName', 'justiceId', 'speakerName', 'text', 'cleanText',
       'start', 'stop', 'duration', 'decidedBy', 'arguedOn', 'petitioner',
       'respondent', 'docket', 'grantedOn', 'decidedOn'],
      dtype='object')

In [150]:
full_df[(full_df['docket'] == '90-1205') & (full_df['justiceId'] == 108)]['duration']

52.23

In [3]:
full_df = pd.read_csv('transcripts_by_justice.csv')

In [4]:
full_df

,term,caseName,justiceId,speakerName,text,cleanText,start,stop,duration,decidedBy,arguedOn,petitioner,respondent,docket,grantedOn,decidedOn
0,1991,United States v. Fordice,108,Clarence Thomas,"Mr. Goodman, is there any distinction between ...",mr goodman be there any distinction between a ...,2186.247,2202.036,15.79,Rehnquist Court,"Nov 13, 1991",United States,Fordice,90-1205,NaN,"Jun 26, 1992"
1,1991,United States v. Fordice,108,Clarence Thomas,"Well, your argument is that you cannot or this...",well your argument be that you can not or this...,2212.804,2224.362,11.56,Rehnquist Court,"Nov 13, 1991",United States,Fordice,90-1205,NaN,"Jun 26, 1992"
2,1991,United States v. Fordice,108,Clarence Thomas,And you give as examples Delta State being und...,and you give as example delta state be underfu...,2224.362,2229.654,5.29,Rehnquist Court,"Nov 13, 1991",United States,Fordice,90-1205,NaN,"Jun 26, 1992"
3,1991,United States v. Fordice,108,Clarence Thomas,My question is if a school is underfunded as a...,my question be if a school be underfunded as a...,2229.654,2245.516,15.86,Rehnquist Court,"Nov 13, 1991",United States,Fordice,90-1205,NaN,"Jun 26, 1992"
4,1991,United States v. Fordice,108,Clarence Thomas,So your argument then would have been that 20 ...,so your argument then would have be that year ...,2252.279,2256.011,3.73,Rehnquist Court,"Nov 13, 1991",United States,Fordice,90-1205,NaN,"Jun 26, 1992"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232513,2018,"Rimini Street, Inc. v. Oracle USA, Inc.",112,"Samuel A. Alito, Jr.",And do you have -- do you have an explanation --,and do you have do you have an explanation,3551.480,3554.160,2.68,Case pending,"Jan 14, 2019","Rimini Street, Inc., et al.","Oracle USA, Inc., et al.",17-1625,"Sep 27, 2018",NaN
232514,2018,"Rimini Street, Inc. v. Oracle USA, Inc.",112,"Samuel A. Alito, Jr.",-- for how Justice Nelson awarded attorneys' f...,for how justice nelson award attorney fee there,3554.440,3557.960,3.52,Case pending,"Jan 14, 2019","Rimini Street, Inc., et al.","Oracle USA, Inc., et al.",17-1625,"Sep 27, 2018",NaN
232515,2018,"Rimini Street, Inc. v. Oracle USA, Inc.",113,Sonia Sotomayor,Why did they use -- why did they use the feder...,why do they use why do they use the federal rule,3592.720,3596.680,3.96,Case pending,"Jan 14, 2019","Rimini Street, Inc., et al.","Oracle USA, Inc., et al.",17-1625,"Sep 27, 2018",NaN
232516,2018,"Rimini Street, Inc. v. Oracle USA, Inc.",111,"John G. Roberts, Jr.","Thank you, counsel.",thank you counsel,3650.720,3652.000,1.28,Case pending,"Jan 14, 2019","Rimini Street, Inc., et al.","Oracle USA, Inc., et al.",17-1625,"Sep 27, 2018",NaN


In [36]:
df = pd.DataFrame(columns=['term', 'caseName', 'justiceId', 'speakerName', 'text', 'cleanText',
                      'duration', 'decidedBy', 'arguedOn', 'petitioner', 
                      'respondent', 'docket', 'grantedOn', 'decidedOn'])

justices = ['Clarence Thomas',
         'Brett M. Kavanaugh',
         'Elena Kagan',
         'John G. Roberts, Jr.',
         'Neil Gorsuch',
         'Ruth Bader Ginsburg',
         'Samuel A. Alito, Jr.',
         'Sonia Sotomayor',
         'Stephen G. Breyer']

justice_ids = {'Clarence Thomas' : 108,
                'Brett M. Kavanaugh' : 116,
                'Elena Kagan' : 114,
                'John G. Roberts, Jr.' : 111,
                'Neil Gorsuch' : 115,
                'Ruth Bader Ginsburg' : 109,
                'Samuel A. Alito, Jr.' : 112,
                'Sonia Sotomayor' : 113,
                'Stephen G. Breyer' : 110}

justice_dfs = [df.copy(), df.copy(), df.copy(), df.copy(), df.copy(), df.copy(), df.copy(), df.copy(), df.copy()]

cases = full_df['docket'].unique()
ct = 0
for case in cases:
    ct += 1
    if ct % 100 == 0:
        print(ct)
    for i, justice in enumerate(justices):
        term = full_df[full_df['docket'] == case]['term'].values[0]
        caseName = full_df[full_df['docket'] == case]['caseName'].values[0]
        decidedBy = full_df[full_df['docket'] == case]['decidedBy'].values[0]
        arguedOn = full_df[full_df['docket'] == case]['arguedOn'].values[0]
        petitioner = full_df[full_df['docket'] == case]['petitioner'].values[0]
        respondent = full_df[full_df['docket'] == case]['respondent'].values[0]
        grantedOn = full_df[full_df['docket'] == case]['grantedOn'].values[0]
        decidedOn = full_df[full_df['docket'] == case]['decidedOn'].values[0]

        id_ = justice_ids[justice]
        temp = full_df[(full_df['docket'] == case) & (full_df['justiceId'] == id_)]
        
        text = ''
        for item in temp['text']:
            text += str(item) + ' '
            
        cleanText = ''
        for item in temp['cleanText']:
            cleanText += str(item) + ' '
        
        durations = temp['duration'].values
        duration = 0
        for d in durations:
            if d > 0:
                duration += d
        
        
        
        justice_dfs[i] = justice_dfs[i].append({'term' : term, 
                                                'caseName' : caseName, 
                                                'justiceId' : id_, 
                                                'speakerName' : justice, 
                                                'text' : text, 
                                                'cleanText' : cleanText,
                                                'duration' : duration,
                                                'decidedBy' : decidedBy,
                                                'arguedOn' : arguedOn, 
                                                'petitioner' : petitioner,
                                                'respondent' : respondent,
                                                'docket' : case,
                                                'grantedOn' : grantedOn, 
                                                'decidedOn' : decidedOn},
                                                ignore_index=True)
        
    

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800


In [85]:
full_df

,term,caseName,justiceId,speakerName,text,cleanText,start,stop,duration,decidedBy,arguedOn,petitioner,respondent,docket,grantedOn,decidedOn
0,1991,United States v. Fordice,108,Clarence Thomas,"Mr. Goodman, is there any distinction between ...",mr goodman be there any distinction between a ...,2186.247,2202.036,15.79,Rehnquist Court,"Nov 13, 1991",United States,Fordice,90-1205,NaN,"Jun 26, 1992"
1,1991,United States v. Fordice,108,Clarence Thomas,"Well, your argument is that you cannot or this...",well your argument be that you can not or this...,2212.804,2224.362,11.56,Rehnquist Court,"Nov 13, 1991",United States,Fordice,90-1205,NaN,"Jun 26, 1992"
2,1991,United States v. Fordice,108,Clarence Thomas,And you give as examples Delta State being und...,and you give as example delta state be underfu...,2224.362,2229.654,5.29,Rehnquist Court,"Nov 13, 1991",United States,Fordice,90-1205,NaN,"Jun 26, 1992"
3,1991,United States v. Fordice,108,Clarence Thomas,My question is if a school is underfunded as a...,my question be if a school be underfunded as a...,2229.654,2245.516,15.86,Rehnquist Court,"Nov 13, 1991",United States,Fordice,90-1205,NaN,"Jun 26, 1992"
4,1991,United States v. Fordice,108,Clarence Thomas,So your argument then would have been that 20 ...,so your argument then would have be that year ...,2252.279,2256.011,3.73,Rehnquist Court,"Nov 13, 1991",United States,Fordice,90-1205,NaN,"Jun 26, 1992"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232513,2018,"Rimini Street, Inc. v. Oracle USA, Inc.",112,"Samuel A. Alito, Jr.",And do you have -- do you have an explanation --,and do you have do you have an explanation,3551.480,3554.160,2.68,Case pending,"Jan 14, 2019","Rimini Street, Inc., et al.","Oracle USA, Inc., et al.",17-1625,"Sep 27, 2018",NaN
232514,2018,"Rimini Street, Inc. v. Oracle USA, Inc.",112,"Samuel A. Alito, Jr.",-- for how Justice Nelson awarded attorneys' f...,for how justice nelson award attorney fee there,3554.440,3557.960,3.52,Case pending,"Jan 14, 2019","Rimini Street, Inc., et al.","Oracle USA, Inc., et al.",17-1625,"Sep 27, 2018",NaN
232515,2018,"Rimini Street, Inc. v. Oracle USA, Inc.",113,Sonia Sotomayor,Why did they use -- why did they use the feder...,why do they use why do they use the federal rule,3592.720,3596.680,3.96,Case pending,"Jan 14, 2019","Rimini Street, Inc., et al.","Oracle USA, Inc., et al.",17-1625,"Sep 27, 2018",NaN
232516,2018,"Rimini Street, Inc. v. Oracle USA, Inc.",111,"John G. Roberts, Jr.","Thank you, counsel.",thank you counsel,3650.720,3652.000,1.28,Case pending,"Jan 14, 2019","Rimini Street, Inc., et al.","Oracle USA, Inc., et al.",17-1625,"Sep 27, 2018",NaN
